In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from numpy import mean, histogram2d
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# 1: Initial Static Heatmap Visualizations

Data is experimental january 1981 body temperature data. 

In [2]:
DATAFILE = "../data/jan1981To.csv"


In [3]:
def readTempData(file):
    from shapely.geometry import Point
    _tdf = pd.read_csv(file, parse_dates=['datetimef'])
    _tdf = _tdf.set_index('datetimef')
    geometry = [Point(d['lon'], d['lat']) for _, d in _tdf.iterrows()]
    datetime = []
    _gdf = gpd.GeoDataFrame(_tdf, geometry = geometry)
    return(_gdf)


In [4]:
tempdata = readTempData(DATAFILE)

In [56]:
def heatmap(df, projection = ccrs.PlateCarree(), interp=None):
    """
        Make heatmap of To_lizard on map given projection and date which is either 'random'
        or a datetime object. 
    """
    ax = plt.axes(projection=projection)
    ax.gridlines(draw_labels=True)
   
    # bounds 
    bounds = df.total_bounds
    ax.set_extent([bounds[0], bounds[2], bounds[1], bounds[3]])

    # extract data 
    grid = histogram2d(df['lon'], df['lat'], bins=(len(df['lon']), len(df['lat'])), weights=df['To_Lizard'])
    
    # interpolate 
    if interp is not None:
        from scipy.interpolate import interp2d
        print(len(grid[1]) * len(grid[2]) ,grid[0].shape)
        interpf  = interp2d(grid[1], grid[2], grid[0], kind = interp)
    # heatmap
    
    ax.pcolormesh(grid[1], grid[2], grid[0], transform=projection)
    
    plt.show()

In [32]:
def map_points(df, lat_col='lat', lon_col='lon', zoom_start=11, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lat_col: Column containing latitude (string)
        lon_col: Column containing longitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()

    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=pt_radius,
                                popup=row['name'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        stations = df[cols_to_pull].as_matrix().tolist()
        curr_map.add_child(plugins.HeatMap(stations, radius=heat_map_radius))

    return curr_map

In [12]:
selected = tempdata.loc['1981-01-01 12:00']
grid = histogram2d(selected['lon'], selected['lat'], bins=(len(selected['lon']), len(selected['lat'])), weights=selected['To_Lizard'].values)